# Import Libraries

In [2]:
import sys
sys.path.append("/home/felix/PycharmProjects/Quantum-Challenge/")
import pandas as pd
import xarray as xr
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import traceback
import math
import networkx as nx
import itertools
from qiskit.algorithms import AmplificationProblem
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover
from qiskit.circuit.library import GroverOperator
from qiskit.extensions import Initialize
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.visualization import plot_histogram
from itertools import product 
from qiskit.quantum_info import Statevector
from importlib import reload
from pytket.circuit import Circuit
from pytket.extensions.qiskit import AerStateBackend, AerBackend  
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.results import probs_from_state  
from qiskit.quantum_info import partial_trace, entropy, Statevector  
import copy
import utils.utils as ut

In [3]:
flight_df = pd.read_csv("../data/flights.csv", sep=";")
cruise_df = pd.read_pickle("../data/cruise_df.pkl")
climb_df = pd.read_pickle("../data/climb_df.pkl")
descent_df = pd.read_pickle("../data/descent_df.pkl")
climate_df = pd.read_pickle("../data/climate_df.pkl")
q_traj_arr = np.load("../data/c_climate_trajectory_arr.npy", allow_pickle=True)

# De-Conflicting

In [4]:
flight_df["cost"] = [ut.C(traj) for traj in q_traj_arr]

In [5]:
unique_coord_dict = ut.unique_coord(q_traj_arr)
confl_list = ut.get_confl_arr(unique_coord_dict)
choice_inx = np.random.choice(np.arange(len(confl_list)),size=5)
confl_arr = confl_list[choice_inx]

# Quantum Circuit

In [15]:
def ansatz(params):
    c = Circuit(n_qubits,0)
    params = params / np.pi  # pytket works in units of pi: Ry(p, q) = exp(-i pi p/2 Y_q)    c = Circuit(n_qubits)
    param_index = 0
    for q in range(n_qubits):
        c.Ry(params[param_index], q)
        param_index += 1
    for l in range(n_layers):
        for q in range(n_qubits-1):
            c.CX(q, q+1)
        for q in range(n_qubits):
            c.Ry(params[param_index], q)
            param_index += 1
    return c

n_qubits = 4
n_layers = 1  # number of ansatz circuit layers
n_params = n_qubits * (1 + n_layers)  

# Defining F-VQE prerequisites

This section defines the auxiliary functions of the F-VQE "protocol". I have tried to arrange them in descending order of importance.
As mentioned earlier, an important component of the F-VQE is the filtering operator $f$. It is defined as a real function $f(c)$, with $c$ as a real parameter, where $f^2(c)$ is strictly decreasing. Furthermore, the $\tau>0$ function parameter can be varied at each optimization step.
Here, an inverse filter was chosen because it showed the best performance in the F-VQE study.

In [16]:
# Filter: function f: c real to f(c) real such that f^2(c) strictly decreases

 # t>0 is a function parameter. Could be changed at each optimization step

def f(c, t):
    
    if np.isclose(c,0):
        return (c+e-5) ** (-t)    
  
    try:
        return np.float64(c) ** np.float64(-t)
    except:
        return (c+e-5) ** (-t+ e-5)

The $\tau$ parameter is continuously adjusted to keep the gradient norm as close as possible to a desired large and fixed value to prevent the gradient from disappearing. 
 
In this case, the implementation is done using a simple heuristic diretly taken from the F-VQE paper.
At each iteration step, we want to choose $\tau$ such that the gradient norm is as close as possible but below a certain threshold $r$, i.e., we solve the implicit equation $\text{gradient_norm}(\tau) = r$


For this we evaluate the gradient norm for increasing values
of $\tau$ until an upper bound is found such that $\text{gradient_norm}(t_{up}) >= r$.
Then we search for $\tau$ in the range $[0, \tau_{up}]$ up to
a certain precision. 

Gradient where probs_plus/minus represents theta shifted by $\pm \frac{\pi}{2}$.
The partial derivative of a parameter is the difference 
between two circuits that differ in a positive/negative $\frac{\pi}{2}$
shift of the parameter. 

In [17]:
# Gradient where probs_plus/minus represents theta shifted by +- pi/2
# The partial derivative of a parameter is the difference 
# between two circuits that differ in a positive/negative pi/2 
# shift of the parameter. 
def g(probs_plus_all, probs_minus_all, t, spectrum):  
    output = np.zeros(n_params)
    for i in range(n_params):
        filter_exp_plus = evaluate_filter(probs_plus_all[i], t, spectrum)
        filter_exp_minus = evaluate_filter(probs_minus_all[i], t, spectrum)
        output[i] = filter_exp_minus - filter_exp_plus
    return output

def t_upper_bound(probs_plus_all, probs_minus_all, spectrum, n_trials=5):
    for t in (3**i for i in range(n_trials)):
        grad_norm = np.linalg.norm(g(probs_plus_all, probs_minus_all, t, spectrum))
        if grad_norm > r:
            return t, grad_norm
    print('Upper bound of t not found. Returning the largest value tested.' + \
          'Increase the number of trials or reduce the threshold for improvement')
    return t, grad_norm

# Once the upper bound is found, we look for the value of t in the range[0, t_up] that is closest and below r
def update_tau(probs_plus_all, probs_minus_all, spectrum):
    precision=r/10
    t_low = 0
    t_up, grad_norm = t_upper_bound(probs_plus_all, probs_minus_all, spectrum)
    while True:
        t = (t_up + t_low) / 2
        grad = g(probs_plus_all, probs_minus_all, t, spectrum)
        grad_norm = np.linalg.norm(grad)
        if r < grad_norm:
            t_up = t
        elif grad_norm < r - precision:
            t_low = t
        else:
            return t, grad

In [18]:
def sample_shifted_ansatz(params):  
    probs_plus_all = {}
    probs_minus_all = {}
    for i in range(n_params):
        params_plus = params + np.bincount([i], None, n_params) * np.pi / 2
        params_minus = params - np.bincount([i], None, n_params) * np.pi / 2
        
        probs_plus = probs_from_state(get_state(params_plus))
        probs_minus = probs_from_state(get_state(params_minus))

        probs_plus_all[i] = probs_plus
        probs_minus_all[i] = probs_minus
    return probs_plus_all, probs_minus_all

def sample_ansatz(params, state):
    c = ansatz(params)
    probs = probs_from_state(state)
    return probs

def get_state(params):
    c = ansatz(params)
    b = AerStateBackend()
    return b.get_compiled_circuit(c).get_statevector()

def update_spectrum(probs, spectrum,cost_function):
    for bitstring, p in probs.items():
        if bitstring in spectrum:
            cost = spectrum[bitstring]
        else:
            cost = cost_function(bitstring, confl_arr, unique_coord_dict)
            spectrum[bitstring] = cost
    return spectrum

def update_params(params, gradient):
    return params - learning_rate * gradient

lower_bound = 0 # min cost
upper_bound = 4 # max cost
def rescale(c, lower_bound=lower_bound, upper_bound=upper_bound):
    b = 1 / (upper_bound - lower_bound)
    a = - lower_bound / (upper_bound - lower_bound)
    return a + b * c

def evaluate_filter(probs, t, spectrum):
    output = 0
    for bitstring, p in probs.items():
        cost = spectrum[bitstring]
        cost = rescale(cost)
        output += f(cost, t) * p
    return output

def evaluate_cost(probs, spectrum):
    output = 0
    for bitstring, p in probs.items():
        cost = spectrum[bitstring]
        output += cost * p
    return output

def get_entanglement(state):
    Q = 0
    for i in range(n_qubits):
        trace_over = [q for q in range(n_qubits) if q != i]  
        reduced_density_matrix = partial_trace(state, trace_over).data  
        #raise matrix to power 2 and trace it
        Q += np.real((np.linalg.matrix_power(reduced_density_matrix, 2)).trace())
    Q = 2 - (2 / n_qubits) * Q
    return Q

def update_print_evolution(probs, t, spectrum, evolution, state):  
    best_cost = min(spectrum.values())
    best_solutions = {bitstring for bitstring, cost in spectrum.items() if cost==best_cost}
    solution = next(iter(best_solutions))
    try: 
        overlap_ground = probs.get(solution, 0)
    except:
        overlap_ground = None
    average_cost = evaluate_cost(probs, spectrum)
    most_likely_solution = max(probs, key=probs.get)
    overlap_most_lik_sol = probs[most_likely_solution]
    evolution['overlap ground state'].append(overlap_ground)
    evolution['best cost'].append(best_cost)
    evolution['average cost'].append(average_cost)
    evolution['overlap most likely solution'].append(overlap_most_lik_sol)
    evolution['tau'].append(t)

    entanglement = get_entanglement(state)
    evolution['entanglement'].append(entanglement)
        
    string_to_print = 'ov gr = ' + '{:.1e}'.format(overlap_ground) +  ', av cost = ' + '{:.1e}'.format(average_cost) + ', best cost = ' + '{:.1e}'.format(best_cost) + ', ent = ' + '{:.1e}'.format(entanglement) 
                           
    return evolution, string_to_print    

def plot_evolution(evolution):
    instructions_l_axis = {'overlap ground state': {'color': 'green'},
                           'overlap most likely solution': {'color': 'blue', 'linestyle': '--'}}

    instructions_l_axis['entanglement'] = {'color': 'gray'}
        
    instructions_r_axis = {'best cost': {'color': 'red', 'linestyle': '--'},
                           'average cost': {'color': 'red'}}
    fig, ax = plt.subplots()
    fig.patch.set_facecolor('white')  
    plt.title(f'{n_qubits} qubits')
    X = np.arange(n_steps + 1)
    plt.xlim(0, n_steps)
    plt.xlabel('optimisation step')
    
    ax.set_ylabel('overlap and entanglement')
    ax.set_ylim([0, 1])
    for label, style in instructions_l_axis.items():
        Y = np.array(evolution[label])
        if style.get('rescale', False): 
            rescale_factor = max([y for y in Y if not np.isnan(y)]) / ax.get_ylim()[1]
            if rescale_factor > 1:
                rescale_factor = 10 ** np.ceil(np.log10(rescale_factor))
                Y /= rescale_factor
                label += ' x ~' + '{:.0e}'.format(rescale_factor)
            del style['rescale']
        ax.plot(X, Y, **style, label=label)
    
    ax2 = ax.twinx()
    ax.plot([], [], color='red', linewidth=3)
    ax2.set_ylabel('cost', color='tab:' + 'red')
    ax2.tick_params(axis='y', labelcolor='tab:' + 'red')
    for label, style in instructions_r_axis.items():
        ax2.plot(X, np.array(evolution[label]), **style, label=label)
    
    ax.legend(bbox_to_anchor=(1.1, 1), loc='upper left')
    ax2.legend(bbox_to_anchor=(1.1, 0.7), loc='upper left')
    plt.show()
    
def fvqe(params, t, n_steps, spectrum, cost_function,allowPrint=True):
    evolution = {'overlap ground state': [], 'best cost': [], 'average cost': [], 
                 'overlap most likely solution': [], 'tau': [], 'entanglement':[]}    
    for step in range(n_steps):   
        state = get_state(params)
        probs = sample_ansatz(params, state) 
        spectrum = update_spectrum(probs, spectrum,cost_function)
        evolution, string_to_print = update_print_evolution(probs, t, spectrum, evolution, state)  
        if allowPrint:
            print(f"step {step}: " + string_to_print)   
        probs_plus_all, probs_minus_all = sample_shifted_ansatz(params)    
        for i in range(n_params):
            spectrum = update_spectrum(probs_plus_all[i], spectrum,cost_function)
            spectrum = update_spectrum(probs_minus_all[i], spectrum,cost_function)
        
        t, gradient = update_tau(probs_plus_all, probs_minus_all, spectrum)
        params = update_params(params, gradient)
        
    step = n_steps  
    probs = sample_ansatz(params,state)  
    spectrum = update_spectrum(probs, spectrum,cost_function)
    evolution, string_to_print = update_print_evolution(probs, t, spectrum, evolution, state)  
    if allowPrint:
        print(f"step {step}: " + string_to_print)  
    return evolution, params, spectrum

In [19]:
original_cost = np.sum([ut.C(tr)for tr in q_traj_arr])

In [20]:
traj_a = q_traj_arr
df = flight_df
def vqe_cost(bit_string, confl_arr, un_coord_dict):
    """ Get cost for vqe """
    traj_arr = traj_a[:]
    p = 0.1
    de_conflicted_traj_arr = ut.bitstr_to_traj(bit_string, confl_arr, traj_arr, unique_coord_dict)
    improved_cost = ut.get_new_cost(confl_arr, de_conflicted_traj_arr, df)
    penalty = p*ut.penalty_cnt(bit_string, confl_arr, traj_arr, un_coord_dict)
    return 100000*(improved_cost - original_cost)/original_cost + penalty 

In [21]:
for bit in list(itertools.product([0, 1], repeat=4)):
    print(vqe_cost(bit, confl_arr, unique_coord_dict))

15.298892355448707
14.854048391942776
-43.890451415969856


KeyboardInterrupt: 

In [22]:
t = 1  # initial value of tau.
r = 0.1  # threshold for tau scheduling.
learning_rate = 0.1
n_steps = 5 # number of F-VQE iterations

spectrum = dict()

n_qubits = 4
n_layers = 1  # number of ansatz circuit layers
n_params = n_qubits * (1 + n_layers) 

params = np.array(list(np.zeros(n_params - n_qubits)) + list(np.ones(n_qubits) * np.pi / 2))

In [23]:
evolution, final_param, spectrum = fvqe(params, t, n_steps,spectrum,cost_function = vqe_cost, allowPrint=False)
plot_evolution(evolution)

/tmp/ipykernel_14604/1057794213.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return np.float64(c) ** np.float64(-t)


RuntimeError: Error trying to simulate circuit tk1(0.5, -nan, 3.5) q[0];
tk1(0.5, -nan, 3.5) q[1];
tk1(0.5, -nan, 3.5) q[2];
tk1(0.5, -nan, 3.5) q[3];
CX q[0], q[1];
tk1(0.5, -nan, 3.5) q[0];
CX q[1], q[2];
tk1(0.5, -nan, 3.5) q[1];
CX q[2], q[3];
tk1(0.5, -nan, 3.5) q[2];
tk1(0.5, -nan, 3.5) q[3];
Phase (in half-turns): 0.0
 with 4 qubits, 11 commands; U is size 16x16, premultiplying M with 16 rows, 1 cols: GateUnitaryMatrix for op tk1(0.5, -nan, 3.5) acting on 1 qubits, taking 3 parameters:
param[0] = 0.5
param[1] = 0
param[2] = 0
parameter[1] has non-finite value -nan

In [118]:
# F-VQE algorithm
    
# initialises the quantum computer in the equal superposition

def fvqe(params, n_steps, un_coord_dict):
    spectrum = {}
    for step in range(n_steps):
        gradient, spectrum = g(params, t, spectrum, un_coord_dict)
        
        params = update_params(params, gradient)
      
        best_cost = min(spectrum.values())
        best_solutions = {bitstring for bitstring, cost in spectrum.items() if cost==best_cost}
        print(f"step {step}: best cost = {best_cost} for {best_solutions}", "\n")
    return spectrum, best_cost, best_solutions

In [119]:
params = np.array(list(np.zeros(n_params - n_qubits)) + list(np.ones(n_qubits) * np.pi / 2)) 
spectrum, best_cost, best_solutions = fvqe(params,  2, unique_coord_dict)

step 0: best cost = 0.011292376673989779 for {(0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0)} 

step 1: best cost = 0.011292376673989779 for {(0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0)} 



In [34]:
# from copy import deepcopy
# def bitstr_to_traj(bit_string, confl_arr, input_traj, un_coord_dict):
#     """ Take bitstring and return changed trajectory.
#     The bit value changed the flight altitude by +-20FL at the confilct point."""
#     traj = deepcopy(input_traj)
#     for bit, confl in zip(bit_string, confl_arr):
#         changed_traj = binary_to_traj(bit, confl, traj, un_coord_dict)
#     return changed_traj


# def binary_to_traj(b_val, conflict_point, input_traj, un_coord_dict):
#     """ Take binary value and translate it to changed trajectory array """
#     traj = deepcopy(input_traj)

#     confl_tuple_arr = []
#     for point in traj[conflict_point[0]]:
#         x = point["x"]
#         y = point["y"]
#         z = point["z"]
#         confl_tuple_arr.append(ut.xyz_to_tuple(x, y, z))

#     i = 0
#     continuous_confl = True
#     while continuous_confl:
#         try:
#             new_confl_cnt = ut.penalty_cnt_bit((b_val,), [[conflict_point[0], conflict_point[1] + i]], input_traj,
#                                             un_coord_dict)
#         except:
#             continuous_confl = False
#             i -= 1
#             break

#         if new_confl_cnt < 0:
#             i += 1
#         else:
#             continuous_confl = False
#             break
#     for cnt, point in enumerate(confl_tuple_arr[conflict_point[1]:conflict_point[1] + i + 1]):
#         confl_tuple_arr[conflict_point[1] + cnt] = (point[0], point[1], point[2] - 1 + b_val * 2)

#     traj[conflict_point[0]] = ut.tuple_path_to_trajec(confl_tuple_arr, conflict_point[0])
#     return traj

In [32]:
def count_confl(x, y, z, un_coord_dict):
    try:
        v = un_coord_dict[str(int(x)) + "_" + str(int(y)) + "_" + str(int(z))]
    except:
        return 0
    cnt = 0
    for inx in range(len(v)):
        for inx_2 in range(inx, len(v)):
            time_diff = np.abs(v[inx][0] - v[inx_2][0])
            if time_diff < v[inx][1]:
                cnt += 1
                print(v[inx])
    return cnt
count_confl(-4, 38, 320, unique_coord_dict)

[numpy.datetime64('2018-06-23T08:28:22'), numpy.timedelta64(726,'s'), 11, 10]
[numpy.datetime64('2018-06-23T09:02:33'), numpy.timedelta64(726,'s'), 18, 7]
[numpy.datetime64('2018-06-23T09:02:33'), numpy.timedelta64(726,'s'), 18, 7]
[numpy.datetime64('2018-06-23T09:02:33'), numpy.timedelta64(726,'s'), 18, 7]
[numpy.datetime64('2018-06-23T09:02:33'), numpy.timedelta64(726,'s'), 18, 7]
[numpy.datetime64('2018-06-23T09:02:33'), numpy.timedelta64(726,'s'), 18, 7]
[numpy.datetime64('2018-06-23T08:53:53'), numpy.timedelta64(726,'s'), 29, 9]
[numpy.datetime64('2018-06-23T09:41:11'), numpy.timedelta64(726,'s'), 43, 9]
[numpy.datetime64('2018-06-23T09:41:11'), numpy.timedelta64(726,'s'), 43, 9]
[numpy.datetime64('2018-06-23T09:41:11'), numpy.timedelta64(726,'s'), 43, 9]
[numpy.datetime64('2018-06-23T09:41:11'), numpy.timedelta64(726,'s'), 43, 9]
[numpy.datetime64('2018-06-23T09:12:05'), numpy.timedelta64(726,'s'), 47, 7]
[numpy.datetime64('2018-06-23T09:12:05'), numpy.timedelta64(726,'s'), 47, 7

27

In [25]:
solution = copy.deepcopy(best_solutions).pop()
improv_traj_arr = ut.bitstr_to_traj(solution, confl_arr, q_traj_arr,unique_coord_dict)
unique_coord_dict_new = ut.unique_coord(improv_traj_arr)
new_confl = ut.get_confl_arr(unique_coord_dict_new)
len(new_confl)

2556

In [ ]:
options = {'backend_name': 'ibmq_guadalupe'}

runtime_inputs = {

  'ansatz': None, 
	
	# A list or dict (with
  # strings as keys) of operators
  # of type PauliSumOp to be
  # evaluated at the final, optimized
  # state.
  'aux_operators': None, # array
	
	# Initial position of virtual qubits
  # on the physical qubits of
  # the quantum device. Default is
  # None.
  'initial_layout': None, # [null,array,object]
	
	# Initial parameters of the ansatz.
  # Can be an array or
  # the string ``'random'`` to choose
  # random initial parameters.
  'initial_parameters': None, # [array,string] (required)
	
	# Whether to apply measurement error
  # mitigation in form of a
  # complete measurement fitter to the
  # measurements. Defaults to False.
  'measurement_error_mitigation': False, # boolean
	
	# The Hamiltonian whose smallest eigenvalue
  # we're trying to find. Should
  # be PauliSumOp
  'operator': None, # object (required)
	
	# The classical optimizer used in
  # to update the parameters in
  # each iteration. Can be either
  # any of Qiskit's Optimizer classes.
  # If a dictionary, only SPSA
  # and QN-SPSA are supported and
  # the dictionary must specify the
  # name and options of the
  # optimizer, e.g. ``{'name': 'SPSA', 'maxiter':
  # 100}``.
  'optimizer': None, # object (required)
	
	# The number of shots used
  # for each circuit evaluation. Defaults
  # to 1024.
  'shots': 1024 # integer
}

IBMQ.load_account()
provider = IBMQ.get_provider(
	hub='deloitte-event',
	group='finalist',
	project='reczjy7jpzcfdlq0'
)

job = provider.runtime.run(
    program_id='vqe',
    options=options,
    inputs=runtime_inputs)

# Job id
print(job.job_id())
# See job status
print(job.status())

# Get results
result = job.result()

Take a state with minimal cost and compute the deconflicted flight schedule.